In [4]:
import os, sys
import numpy as np
import pandas as pd
import cv2
#import matplotlib.pyplot as plt
#import seaborn as sns
import json
import mediapipe as mp
from IPython.display import Image

%matplotlib inline #notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것

UsageError: unrecognized arguments: #notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것


## mp_train/push_up/down_correct/ 의 파일들 각도 계산

In [5]:
path = './data/mp_train/push_up/down_correct/'  # 라벨링 데이터의 경로 지정
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.png')] # 해당 경로의 json 파일들을 목록화

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

df=pd.DataFrame(columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨'])
for i in file_list_py:
    image = cv2.imread(path+i, cv2.IMREAD_COLOR)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angle
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_shoulder_angle=calculate_angle(left_hip, left_shoulder, left_elbow)

            # Get coordinates
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate angle
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_shoulder_angle=calculate_angle(right_hip, right_shoulder, right_elbow)


        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        """cv2.namedWindow('Mediapipe Feed', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Mediapipe Feed', 1000, 750)
        cv2.imshow('Mediapipe Feed', image)"""
        """print(i)
        print('왼쪽 팔꿈치 : ', left_elbow_angle)
        print('왼쪽 어깨 : ', left_shoulder_angle)
        print('오른쪽 팔꿈치 : ', right_elbow_angle)
        print('오른쪽 어깨 : ', right_shoulder_angle)"""

        k = pd.DataFrame(data=[[left_elbow_angle, left_shoulder_angle, right_elbow_angle, right_shoulder_angle, 1]], 
                         columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨', '점수'])
        df=df.append(k)
        df = df.reset_index(drop=True)

        #print()
        """cv2.waitKey(0)
        cv2.destroyAllWindows()"""

df

,왼쪽 팔꿈치,왼쪽 어깨,오른쪽 팔꿈치,오른쪽 어깨,점수
0,36.094779,5.313295,20.662132,66.933597,1.0
1,41.106454,3.425422,15.317009,75.770951,1.0
2,42.467221,0.600137,11.236330,73.744296,1.0
3,55.117431,16.374145,9.676304,65.725450,1.0
4,39.050797,5.651023,11.003275,69.046638,1.0
5,41.166219,1.474979,13.780750,73.694411,1.0
6,34.780610,12.848503,7.391771,63.785820,1.0
7,54.788232,28.225031,24.300288,70.585297,1.0
8,51.819262,29.544420,21.054014,61.264866,1.0
9,39.688166,12.868043,1.838766,57.210561,1.0


## mp_train/push_up/down_ambiguous/ 의 파일들 각도 계산

In [6]:
path = './data/mp_train/push_up/down_ambiguous/'  # 라벨링 데이터의 경로 지정
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.png')] # 해당 경로의 json 파일들을 목록화

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

#df=pd.DataFrame(columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨'])
for i in file_list_py:
    image = cv2.imread(path+i, cv2.IMREAD_COLOR)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angle
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_shoulder_angle=calculate_angle(left_hip, left_shoulder, left_elbow)

            # Get coordinates
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate angle
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_shoulder_angle=calculate_angle(right_hip, right_shoulder, right_elbow)


        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        k = pd.DataFrame(data=[[left_elbow_angle, left_shoulder_angle, right_elbow_angle, right_shoulder_angle, 1]], 
                         columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨', '점수'])
        df=df.append(k)
        df = df.reset_index(drop=True)

df

,왼쪽 팔꿈치,왼쪽 어깨,오른쪽 팔꿈치,오른쪽 어깨,점수
0,36.094779,5.313295,20.662132,66.933597,1.0
1,41.106454,3.425422,15.317009,75.770951,1.0
2,42.467221,0.600137,11.236330,73.744296,1.0
3,55.117431,16.374145,9.676304,65.725450,1.0
4,39.050797,5.651023,11.003275,69.046638,1.0
5,41.166219,1.474979,13.780750,73.694411,1.0
6,34.780610,12.848503,7.391771,63.785820,1.0
7,54.788232,28.225031,24.300288,70.585297,1.0
8,51.819262,29.544420,21.054014,61.264866,1.0
9,39.688166,12.868043,1.838766,57.210561,1.0


## mp_train/push_up/down_wrong/ 의 파일들 각도 계산

In [7]:
path = './data/mp_train/push_up/down_wrong/'  # 라벨링 데이터의 경로 지정
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.png')] # 해당 경로의 json 파일들을 목록화

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

#df=pd.DataFrame(columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨'])
for i in file_list_py:
    image = cv2.imread(path+i, cv2.IMREAD_COLOR)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angle
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_shoulder_angle=calculate_angle(left_hip, left_shoulder, left_elbow)

            # Get coordinates
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate angle
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_shoulder_angle=calculate_angle(right_hip, right_shoulder, right_elbow)


        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        k = pd.DataFrame(data=[[left_elbow_angle, left_shoulder_angle, right_elbow_angle, right_shoulder_angle, 0]], 
                         columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨', '점수'])
        df=df.append(k)
        df = df.reset_index(drop=True)


df

,왼쪽 팔꿈치,왼쪽 어깨,오른쪽 팔꿈치,오른쪽 어깨,점수
0,36.094779,5.313295,20.662132,66.933597,1.0
1,41.106454,3.425422,15.317009,75.770951,1.0
2,42.467221,0.600137,11.236330,73.744296,1.0
3,55.117431,16.374145,9.676304,65.725450,1.0
4,39.050797,5.651023,11.003275,69.046638,1.0
...,...,...,...,...,...
56,166.541147,42.530879,143.911683,39.343384,0.0
57,161.994109,29.581399,150.140006,27.410511,0.0
58,176.265904,22.840737,126.416010,45.434912,0.0
59,133.109485,60.540895,102.234927,68.452590,0.0


## mp_train/squat/down_correct/ 의 파일들 각도 계산

In [19]:
path = './data/mp_train/squat/down_correct/'  # 라벨링 데이터의 경로 지정
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.png')] # 해당 경로의 json 파일들을 목록화

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

df2=pd.DataFrame(columns=['왼쪽 허리', '왼쪽 무릎', '오른쪽 허리', '오른쪽 무릎', '점수'])
for i in file_list_py:
    image = cv2.imread(path+i, cv2.IMREAD_COLOR)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            # Calculate angle
            left_waist_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            left_knee_angle=calculate_angle(left_hip, left_knee, left_ankle)

            # Get coordinates
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Calculate angle
            right_waist_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            right_knee_angle=calculate_angle(right_hip, right_knee, right_ankle)


        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        k = pd.DataFrame(data=[[left_waist_angle, left_knee_angle, right_waist_angle, right_knee_angle, 1]], 
                         columns=['왼쪽 허리', '왼쪽 무릎', '오른쪽 허리', '오른쪽 무릎', '점수'])
        df2=df2.append(k)
        df2 = df2.reset_index(drop=True)

df2

,왼쪽 허리,왼쪽 무릎,오른쪽 허리,오른쪽 무릎,점수
0,157.824640,164.332007,157.085869,163.675292,1
1,146.525138,153.279985,155.303240,160.519401,1
2,154.139951,161.275129,159.490803,165.923463,1
3,150.133035,156.282793,157.069828,163.770520,1
4,144.087121,150.929741,146.934441,153.940448,1
5,149.503825,154.903091,159.180758,166.164652,1
6,155.204424,163.100170,160.108102,168.290073,1
7,162.022142,169.269675,149.047493,157.714857,1
8,155.925110,163.818021,153.803909,160.753852,1
9,154.667032,160.987056,160.093807,166.316481,1


## mp_train/squat/down_wrong/ 의 파일들 각도 계산

In [21]:
path = './data/mp_train/squat/down_wrong/' # 라벨링 데이터의 경로 지정
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.png')] # 해당 경로의 json 파일들을 목록화

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

#df=pd.DataFrame(columns=['왼쪽 팔꿈치', '왼쪽 어깨', '오른쪽 팔꿈치', '오른쪽 어깨'])
for i in file_list_py:
    image = cv2.imread(path+i, cv2.IMREAD_COLOR)

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            # Calculate angle
            left_waist_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            left_knee_angle=calculate_angle(left_hip, left_knee, left_ankle)

            # Get coordinates
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Calculate angle
            right_waist_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            right_knee_angle=calculate_angle(right_hip, right_knee, right_ankle)


        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        k = pd.DataFrame(data=[[left_waist_angle, left_knee_angle, right_waist_angle, right_knee_angle, 0]], 
                         columns=['왼쪽 허리', '왼쪽 무릎', '오른쪽 허리', '오른쪽 무릎', '점수'])
        df2=df2.append(k)
        df2 = df2.reset_index(drop=True)

df2

,왼쪽 허리,왼쪽 무릎,오른쪽 허리,오른쪽 무릎,점수
0,157.824640,164.332007,157.085869,163.675292,1
1,146.525138,153.279985,155.303240,160.519401,1
2,154.139951,161.275129,159.490803,165.923463,1
3,150.133035,156.282793,157.069828,163.770520,1
4,144.087121,150.929741,146.934441,153.940448,1
5,149.503825,154.903091,159.180758,166.164652,1
6,155.204424,163.100170,160.108102,168.290073,1
7,162.022142,169.269675,149.047493,157.714857,1
8,155.925110,163.818021,153.803909,160.753852,1
9,154.667032,160.987056,160.093807,166.316481,1


# df와 df2를 tsv파일로

# 모델 학습 (Logistic Regression)
### 일단 팔굽혀펴기(df) 만

In [8]:
#기본모듈
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#전처리 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#머신러닝 모델 라이브러리
from sklearn.linear_model import LogisticRegression

#평가지표

from sklearn.metrics import accuracy_score, recall_score, precision_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import classification_report

In [9]:
X=df.drop(['점수'], axis=1)
y=df['점수']

In [10]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=2/10, stratify=y, random_state=12124343)

In [11]:
LR=LogisticRegression(penalty='none', solver='sag', max_iter=5000, tol=1e-4) 
# solver에 gradient descent 같은 알고리즘을 넣는 것
#LR=LogisticRegression()
#LR=LogisticRegression(penalty='l2', solver='newtone-cg', max_iter=3000, tol=1e-4, C=1.5)
# Newton-CG methods use a Conjugate Gradient

In [12]:
LR.fit(X_train, y_train)

LogisticRegression(max_iter=5000, penalty='none', solver='sag')

In [13]:
y_train_pred=LR.predict(X_train)
y_test_pred=LR.predict(X_test)

In [14]:
print('- Accuracy(Test) : {:.4}'.format(accuracy_score(y_test, y_test_pred)))

print('- precision score(Test) : {:.4}'.format(precision_score(y_test, y_test_pred, pos_label=0))) # positive 라벨이 무엇인지 지정해준 것

print('- recall score(Test) : {:.4}'.format(recall_score(y_test, y_test_pred, pos_label=0))) # 지정하지 않으면 1로 지정되어서

print('- F1 score(Test) : {:.4}'.format(f1_score(y_test, y_test_pred, pos_label=0))) # 에러가 나올 걸

print('- balanced score(Test) : {:.4}'.format(balanced_accuracy_score(y_test, y_test_pred)))

- Accuracy(Test) : 1.0
- precision score(Test) : 1.0
- recall score(Test) : 1.0
- F1 score(Test) : 1.0
- balanced score(Test) : 1.0


# 모델 저장 및 예측

In [16]:
import pickle
#from sklearn.externals import joblib
saved_model = pickle.dumps(LR)
#joblib.dump(clf, 'filename.pkl') 

In [17]:
clf_from_pickle = pickle.loads(saved_model)

In [18]:
clf_from_pickle.predict([[23, 124, 412, 1], [21, 123, 4, 12]])

array([1., 0.])

In [19]:
if clf_from_pickle.predict([[23, 124, 412, 1]])==0:
    print('못')
elif clf_from_pickle.predict([[23, 124, 412, 1]])==1:
    print('잘')

잘


In [20]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import tensorflow as tf
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

model = tf.keras.models.load_model('model.h5')
cap = cv2.VideoCapture(0)

#fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
#out = cv2.VideoWriter('my_count2.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    a=[]
    b=[]
    c=[]
    d=[]
    precise=[]

    ret, frame1 = cap.read()
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255
    i = 0
    prediction_str = ""
    repetitions = 0
    up = 0
    down = 0
    no_move = 0
    current_move = 0
    initial = -1
    # Curl counter variables
    counter = 0
    stage = None
    while cap.isOpened():
        i += 1
        ret, frame2 = cap.read()
        if not ret: break
        if cv2.waitKey(30) >= 0: break
        # Recolor image to RGB
        image = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        image2 = cv2.resize(rgb, (64, 64))
        image2 = image2.reshape((1,) + image2.shape)
        image2 = image2 / 255.0
        prediction = np.argmax(model.predict(image2), axis=-1)[0]

        if prediction == 0:
            down += 1
            if down == 3:
                if initial == -1:
                    initial = 0
                if current_move == 2:
                    counter += 1
                current_move = 0
            elif down > 0:
                up = 0
                no_move = 0
        elif prediction == 2:
            up += 1
            if up == 3 and initial != -1:
                current_move = 2
            elif up > 1:
                down = 0
                no_move = 0
        else:
            no_move += 1
            if no_move == 15:
                current_move = 1
            elif no_move > 10:
                up = 0
                down = 0
        font = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (10, 400)
        fontScale = 1
        fontColor = (255, 255, 255)
        lineType = 5
        #cv2.imshow("wow", frame2)
        #out.write(frame2)
        prvs = next



        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            # Calculate angle
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            left_shoulder_angle=calculate_angle(left_hip, left_shoulder, left_elbow)

            # Get coordinates
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate angle
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            right_shoulder_angle=calculate_angle(right_hip, right_shoulder, right_elbow)
            
            # Curl counter logic
            if left_elbow_angle < 110:
                stage = "down"
                a.append(left_elbow_angle)
                b.append(left_shoulder_angle)
                c.append(right_elbow_angle)
                d.append(right_shoulder_angle)
            if left_elbow_angle > 130 and stage == 'down':
                stage = "up"
                tmp=min(a)
                index=a.index(tmp)
                if clf_from_pickle.predict([[tmp, b[index], c[index], d[index]]])==1:
                    print("잘했습니다")
                else:
                    print("더 숙이세요")
                a.clear()
                b.clear()
                c.clear()
                d.clear()
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(image, 'STAGE', (65, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (60, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        cv2.namedWindow('Mediapipe Feed', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Mediapipe Feed', 1000, 750)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
더 숙이세요
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
더 숙이세요
ㄴㄴㄴ
ㅋㅋㅋ
더 숙이세요
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
더 숙이세요
ㄴㄴㄴ
ㅋㅋㅋ
더 숙이세요
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㅋㅋㅋ
잘했습니다
ㄴㄴㄴ
ㄴㄴㄴ
ㅋㅋㅋ
더 숙이세요


KeyboardInterrupt: 